In [ ]:
# !pip install langchain milvus pymilvus python-dotenv

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

In [ ]:
import os
from dotenv import load_dotenv
import openai
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from milvus import default_server

In [3]:
default_server.start()



    __  _________ _   ____  ______
   /  |/  /  _/ /| | / / / / / __/
  / /|_/ // // /_| |/ / /_/ /\ \
 /_/  /_/___/____/___/\____/___/ {Lite}

 Welcome to use Milvus!

 Version:   v2.2.8-lite
 Process:   13978
 Started:   2023-05-30 08:34:48
 Config:    /Users/yujiantang/.milvus.io/milvus-server/2.2.8/configs/milvus.yaml
 Logs:      /Users/yujiantang/.milvus.io/milvus-server/2.2.8/logs

 Ctrl+C to exit ...


In [4]:
from langchain.vectorstores import Milvus
embeddings = OpenAIEmbeddings()

In [5]:
# !pip install pymilvus
from pymilvus import utility, connections

connections.connect(host="127.0.0.1", port=default_server.listen_port)
utility.drop_collection('LangChainCollection')

In [6]:
vectordb = Milvus.from_documents(
    {},
    embeddings,
    connection_args={"host": "127.0.0.1", "port": default_server.listen_port})

In [7]:
retriever = Milvus.as_retriever(vectordb, search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [8]:
about_me = [
    {"input": "My favorite snack is chocolate",
     "output": "Nice"},
    {"input": "My favorite sport is swimming",
     "output": "Cool"},
    {"input": "My favorite beer is Guinness",
     "output": "Great"},
    {"input": "My favorite dessert is cheesecake",
     "output": "Good to know"},
    {"input": "My favorite musician is Taylor Swift",
     "output": "Same"}
]
for example in about_me:
    memory.save_context({"input": example["input"]}, {"output": example["output"]})

In [9]:
print(memory.load_memory_variables({"prompt": "who is my favorite musician?"})["history"])

input: My favorite musician is Taylor Swift
output: Same


In [10]:
llm = OpenAI(temperature=0) # Can be any valid LLM
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is Gary, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite snack is chocolate
output: Nice

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, my name is Gary, what's up?
AI:

> Finished chain.


" Hi Gary, my name is AI. I'm doing well. How about you?"

In [12]:
conversation_with_summary.predict(input="who is my favorite musician?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: who is my favorite artist?
response:  You said your favorite musician is Taylor Swift.

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: who is my favorite musician?
AI:

> Finished chain.


' You said your favorite musician is Taylor Swift. Is there anything else I can help you with?'

In [13]:
conversation_with_summary.predict(input="Whats my favorite dessert?")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite dessert is cheesecake
output: Good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Whats my favorite dessert?
AI:

> Finished chain.


' You said your favorite dessert is cheesecake.'

In [14]:
conversation_with_summary.predict(input="What's my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, my name is Gary, what's up?
response:  Hi Gary, my name is AI. I'm doing well. How about you?

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: What's my name?
AI:

> Finished chain.


' You said your name is Gary.'

In [15]:
default_server.stop()